<div class="alert alert-danger">
**Due date:** 2017-02-03
</div>

# Lab 2: Language Modelling

**Students:** Maria Johansson (marjo123), Erik Karlsson (erika456)

## Introduction

In this lab you will experiment with $n$-gram models. You will test various parameters that influence these models&rsquo; quality and train to estimate models with additive smoothing.

The following lines of code import the Python modules needed for this lab:

In [1]:
import lab2
import ngrams

The data for this lab consists of Arthur Conan Doyle&rsquo;s novels about Sherlock Holmes: *The Adventures of Sherlock Holmes*, *The Memoirs of Sherlock Holmes*, *The Return of Sherlock Holmes*, *His Last Bow* and *The Case-Book of Sherlock Holmes*. The next piece of code loads the first three of these as training data:

In [2]:
training_data = lab2.read_data("/home/729G17/labs/lab2/data/advs.txt",
                               "/home/729G17/labs/lab2/data/mems.txt",
                               "/home/729G17/labs/lab2/data/retn.txt")

The data is represented as a list of sentences, where one sentence is represented as a list of tokens (strings). The next line prints the 101th sentence:

In [3]:
print(training_data[100])

["'", 'Let', 'us', 'glance', 'at', 'our', 'Continental', 'Gazetteer', '.']


## Relation between a model’s quality and its order

In the first part of this lab you will examine the relation between an $n$-gram model’s quality and its **order**, i.e. the value of&nbsp;$n$. You will do both a qualitative and quantitative evaluation with the help of the entropy measure.

### Qualitative evaluation

The following line trains a bigram-model of the class `ngrams.Model` on the training data.

In [4]:
model = lab2.train(ngrams.Model, 2, training_data)

With this model you are able to generate random sentences. Every time you run the following code cell a new sentence is generated.

In [5]:
print(" ".join(model.generate()))

Sometimes I am sure that we will trouble you keep the third from a quarter of another cigarette .


Look at the sentences. Do they sound natural?

<div class="panel panel-primary">
<div class="panel-heading">Problem 1</div>
<div class="panel-body">
Train a unigram-, bigram-, trigram-, and quadrigram-model, and generate random sentences with each. How does the quality of the sentences change with the model’s order? Explain your observations using your understanding from how an $n$-gram model works. Use some generated sentences in order to illustrate your discussion. How would the sentences look like for higher values of $n$, such as $n=10$?
</div>
</div>

In [6]:
# TODO: Insert your code here
uni = lab2.train(ngrams.Model, 1, training_data)
bi = lab2.train(ngrams.Model, 2, training_data)
tri = lab2.train(ngrams.Model, 3, training_data)
quad = lab2.train(ngrams.Model, 4, training_data)

print(" ".join(uni.generate()))
print(" ".join(bi.generate()))
print(" ".join(tri.generate()))
print(" ".join(quad.generate()))

ill-gotten . . lady's me we
One was a little disturbed and being insane from death .
An hour and half an hour the miserable weather and the Colonel for five years and was not in a sort of three-card trick , is my own mind as to make it clear ?
Holmes sat down and stretched his legs before the fire and composed himself to listen .


*TODO: Insert your discussion here*

### Quantitative evaluation

In order to do a quantitative evaluation of a model we can compute its **entropy** on held-out data. We will use the first part of the novel *The Adventures of Sherlock Holmes* for this. It is loaded by the following command:

In [7]:
test_data = lab2.read_data("/home/729G17/labs/lab2/data/test.txt")

The next piece of code trains a bigram-model and computes its entropy on the test data:

In [8]:
model = lab2.train(ngrams.Model, 2, training_data)
lab2.evaluate(model, test_data)

3.426862596420277

<div class="panel panel-primary">
<div class="panel-heading">Problem 2</div>
<div class="panel-body">
Compute the entropy for the four models you created for the previous problem. How does the model’s entropy change with the model’s order? Explain using your knowledge of the entropy measure.
</div>
</div>

In [10]:
# TODO: Insert your code here
print(lab2.evaluate(uni, test_data))
print(lab2.evaluate(bi, test_data))
print(lab2.evaluate(tri, test_data))
print(lab2.evaluate(quad, test_data))

7.337551182974018
3.426862596420277
1.4289533769461726
0.5436027106964166


*TODO: Insert your discussion here*

## Relation between a model’s quality and the estimation method

In the second part of this lab you will implement and evaluate various estimation methods. In order to do that you will need to know how the lab system is built up.

### The content of a model

When you call the `train()` function (like you did above), the system creates an $n$-gram model of the given class (so far: `ngrams.Model`) and with the given order (the value of $n$) and trains the model on the given data set. For the second part of this lab you will use your own model class. We start with defining the class in such a way that it simply calls the corresponding methods of the superclass:

In [11]:
class Model(ngrams.Model):
    
    def order(self):
        """Return the order of this model (an integer)."""
        return super().order()
    
    def vocabulary(self):
        """Return this model's vocabulary (a set)."""
        return super().vocabulary()
    
    def freq(self, ctxt, word):
        """Return the number of occurrences of `word` (a string) after `ctxt` (a tuple of strings)."""
        return super().freq(ctxt, word)
    
    def total(self, ctxt):
        """Return the total number of ngrams that start with `ctxt` (a tuple of strings)."""
        return super().total(ctxt)
    
    def prob(self, ctxt, word):
        """Return the probability for `word` (a string) given `ctxt` (a tuple of strings)."""
        return super().prob(ctxt, word)

The next piece of code trains a bigram-model of the class `Model` and prints the model’s order (an integer) and the size of its vocabulary (a set of strings, represented by Python’s `set` type).

In [12]:
model = lab2.train(Model, 2, training_data)
print("order of the model:", model.order())
print("number of words in the model's vocabulary:", len(model.vocabulary()))

order of the model: 2
number of words in the model's vocabulary: 15339


#### Look up an n-gram’s absolute frequency

A trained model consists primarily of a table with absolute frequencies for all $n$-grams that appear in the text it was trained on. In order to look up an $n$-gram’s absolute frequency one can use the method `freq()`. An $n$-gram is divided into two parts: an $(n-1)$-gram called **context** (`ctxt`) and a final unigram (`word`). In Python the context is represented as a tuple of strings and the unigram as a normal string.

If you want to train a trigram model and then know the absolute frequency for the trigram *Mr. Sherlock Holmes* you can write:

In [13]:
model = lab2.train(Model, 3, training_data)
model.freq(("Mr.", "Sherlock"), "Holmes")

50

For training a bigram model and looking up the absolute frequency for the bigram *Baker Street* you can write the following. Note that the context of a bigram model is a 1-tuple of strings, which has a special notation in Python.

In [14]:
model = lab2.train(Model, 2, training_data)
model.freq(("Baker",), "Street")

67

#### Look up the absolute frequency of an n-gram with a given context

The method `total()` returns the absolute frequency of $n$-grams with the given kontext. Here is an example for a trigram model:

In [15]:
model = lab2.train(Model, 3, training_data)
model.total(("Mr.", "Sherlock"))

50

<div class="panel panel-primary">
<div class="panel-heading">Problem 3</div>
<div class="panel-body">
Train a bigram model and use it to calculate the following values, using the methods shown above.
</div>
</div>

In [16]:
model = lab2.train(Model, 2, training_data)

**3.1.** the absolute frequency for the bigram *Sherlock Holmes*

In [22]:
# TODO: Insert your code here
model.total(("Mr."), "Sherlock")

TypeError: total() takes 2 positional arguments but 3 were given

**3.2.** the absolute frequency of bigrams with the context *Sherlock*

In [ ]:
# TODO: Insert your code here

**3.3.** the absolute frequency for the unigram *Sherlock*

In [ ]:
# TODO: Insert your code here

**3.4.** the number of words in the vocabulary

In [ ]:
# TODO: Insert your code here

**3.5.** a list with all the words following the context *Sherlock*

In [ ]:
# TODO: Insert your code here

(For the last exercise you will need to write a bit more than a simple function call.)

### Estimate probabilities with the Maximum Likelihood method

The method `prob()` returns the estimated conditional probability $P(w|c)$ for a word $w$ given a context $c$. The following code snippet trains a trigram model and estimates the pobability for *Holmes* given the context *Mr. Sherlock*:

In [ ]:
model = lab2.train(Model, 3, training_data)
model.prob(("Mr.", "Sherlock"), "Holmes")

(What does the returned value imply?)

<div class="panel panel-primary">
<div class="panel-heading">Problem 4</div>
<div class="panel-body">
Do your own implementation of the method `prob()`. The method should estimate probabilities using the Maximum Likelihood method. Test your implementation by redoing Exercise&nbsp;2 with the new class; you should get the same result as before. Use the code you wrote in Exercise&nbsp;3 in order to solve the exercise.
</div>
</div>

In [ ]:
class Model(ngrams.Model):
    
    def prob(self, ctxt, word):
        """Return the probability for `word` (a string) given `ctxt` (a tuple of strings)."""
        # TODO: Replace the next line with your own code
        return super().prob(ctxt, word)

# TODO: Insert your testing code here

In order to solve this exercise you will need to turn the formula for Maximum Likelihood estimation into code. We illustrate the formula for a bigram model. If we write $f(w_1w_2)$ for the number of occurrences of the bigram  $w_1w_2$ and $f(w_1)$ for the number of occurrences of the unigram $w_1$, then the probability for observing $w_2$ given $w_1$ is
$$
P(w_2|w_1) = \frac{f(w_1w_2)}{f(w_1)}
$$

### Problems with Maximum Likelihood estimation

The file `yoda.txt` contains the same text as `test.txt`, but in the jumbled [Yoda-language]( http://itre.cis.upenn.edu/~myl/languagelog/archives/002173.html).

In [ ]:
yoda_data = lab2.read_data("/home/729G17/labs/lab2/data/yoda.txt")

<div class="panel panel-primary">
<div class="panel-heading">Problem 5</div>
<div class="panel-body">
Redo the evaluation of the four previous models with `yoda.txt` as test data. Something unexpected happens for models with $n>1$. Why? Explain the problem with your knowledge of Maximum Likelihood estimation.
</div>
</div>

In [ ]:
# TODO: Insert your code here

*TODO: Insert your discussion here*

### Estimate probabilities with additive smoothing

For the next problem you are going to do Maximum Likelihood estimation, but with additive smoothing.

<div class="panel panel-primary">
<div class="panel-heading">Problem 6</div>
<div class="panel-body">
<p>
Write a new implementation of the method `prob()`, such that it estimates probabilities with additive smoothing.</p>
<p>
Evaluate the system with new new class using the entropy measure from Problem&nbsp;2. Choose the following values for the smoothing constant $k$:&nbsp;0,00,&nbsp;1,00, 0,10, 0,01. For $k=0$ you should get the same result as in Problem&nbsp;5.
</p>
<p>
How and why does the smoothing constant influence the model’s entropy? Connect to the distribution of the probability mass between observed and imaginary occurrences.
</p>
</div>
</div>

In [ ]:
class Model(ngrams.Model):
    
    def prob(self, ctxt, word):
        """Return the probability for `word` (a string) given `ctxt` (a tuple of strings)."""
        # TODO: Replace the next line with your own code
        return super().prob(ctxt, word)

# TODO: Insert your testing code here

Fill the table below with your entropy measures.

<table>
<tr><td></td><td>k = 0,00</td><td>k = 1,00</td><td>k = 0,10</td><td>k = 0,01</td></tr>
<tr><td>n = 1</td><td>to fill</td><td>to fill</td><td>to fill</td><td>to fill</td></tr>
<tr><td>n = 2</td><td>to fill</td><td>to fill</td><td>to fill</td><td>to fill</td></tr>
<tr><td>n = 3</td><td>to fill</td><td>to fill</td><td>to fill</td><td>to fill</td></tr>
<tr><td>n = 4</td><td>to fill</td><td>to fill</td><td>to fill</td><td>to fill</td></tr>
</table>

*TODO: Insert your discussion here*

### An unseen test set

Your last exercise is to redo the evaluation on a previously unseen test set, texts from the collection *His Last Bow*.

In [ ]:
unseen_data = lab2.read_data("/home/729G17/labs/lab2/data/lstb.txt")

<div class="panel panel-primary">
<div class="panel-heading">Problem 7</div>
<div class="panel-body">
Redo the evaluation from Problem 6 with the new test data. Explain what happens given the differences between `test.txt` and `lstb.txt`.
</div>
</div>

In [ ]:
# TODO: Insert your code here

*TODO: Insert your explanations here*